In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_20.csv')
pred2 = dd.read_csv('test_pred_22.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 33.1 s, sys: 2.92 s, total: 36.1 s
Wall time: 8.11 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 166 ms, sys: 276 ms, total: 442 ms
Wall time: 441 ms


In [5]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 311 µs, sys: 0 ns, total: 311 µs
Wall time: 315 µs


In [6]:
coef_lgb = 0.6
coef_mlp = 0.4

In [7]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 329 ms, sys: 633 ms, total: 962 ms
Wall time: 959 ms


In [8]:
print(test_ensemble.shape)
test_ensemble['object_id'] = pred1['object_id']

(3492890, 15)


In [10]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [11]:
print(test_ensemble.shape)

(3492890, 16)


In [12]:
test_ensemble.head()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
0,13,0.000204,0.000665,0.000077,0.716135,0.063446,0.000245,0.176995,0.000041,0.000073,0.002283,0.000182,0.038528,0.000094,0.001032,0.158510
1,14,0.000100,0.006649,0.000100,0.164192,0.063597,0.000184,0.050466,0.000128,0.000375,0.026309,0.001020,0.679180,0.000102,0.007597,0.180241
2,17,0.000156,0.008025,0.000156,0.114114,0.072401,0.000270,0.024071,0.001430,0.000332,0.102220,0.001914,0.661995,0.000105,0.012812,0.177012
3,23,0.000171,0.001476,0.000180,0.055475,0.037772,0.000244,0.065190,0.003237,0.000262,0.447062,0.000721,0.363399,0.000098,0.024713,0.230726
4,34,0.000039,0.001525,0.000035,0.082050,0.164386,0.000125,0.020151,0.000022,0.000071,0.029621,0.000064,0.700363,0.000036,0.001512,0.172460


In [13]:
%%time
test_ensemble.to_csv('test_pred_24.csv',index=False)

CPU times: user 1min 26s, sys: 1.11 s, total: 1min 27s
Wall time: 1min 28s


In [14]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_24.csv -m "Message"

100%|█████████████████████████████████████| 1.06G/1.06G [2:37:38<00:00, 121kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification